In [1]:
import xarray as xr
import numpy as np
import datetime
import matplotlib.pyplot as plt

from cartopy.crs import PlateCarree
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.mpl.ticker import (LongitudeFormatter, LatitudeFormatter,
                                LatitudeLocator, LongitudeLocator)
import cartopy.feature as cfeature


import matplotlib.ticker as mticker
from matplotlib.cm import get_cmap, ScalarMappable
from matplotlib.colors import Normalize

def make_grid(ax, font_size):
        gl = ax.gridlines(linewidth=1, color='gray', alpha=0.5,
                    linestyle='--',draw_labels=True)
        gl.xlabels_top = False
        gl.ylabels_right = False

        gl.xlabel_style = {'fontsize': font_size}
        gl.ylabel_style = {'fontsize': font_size}

        return gl

def set_plot(figsize = [7,7], nrows = 1, ncols = 1):
    fig, ax = plt.subplots(subplot_kw = dict(projection = PlateCarree()), figsize = figsize, nrows = nrows, ncols = ncols)
    
    font_size = 13
    if nrows == 1 and ncols == 1:
        ax.coastlines(resolution="10m", linewidths=0.5)
        ax.add_feature(cfeature.LAND.with_scale("10m"),
                edgecolor='lightgray',facecolor='lightgray',
                zorder=0)

        ax.tick_params(axis = "both", labelsize = font_size)

        gl = make_grid(ax, font_size)

    else:
        ax = ax.ravel()
        for a in ax:
            a.coastlines(resolution="10m", linewidths=0.5)
            a.add_feature(cfeature.LAND.with_scale("10m"),
                edgecolor='lightgray',facecolor='lightgray',
                zorder=0)

            a.tick_params(axis = "both", labelsize = font_size)
            gl = make_grid(a, font_size)

    return fig, ax

# Unification of MODIS chlorophyll files
# (do not run!)

In [15]:
temp = xr.open_dataset("../data/SCS/CMEMS_temp_extended.nc").analysed_sst

lons, lats = temp.lon, temp.lat

In [4]:
chl = xr.open_mfdataset("../data/SCS/MODIS_DATA_DAILY/AQUA_MODIS.*.L3m.DAY.CHL.x_chlor_a.nc", combine = "nested", concat_dim="time")

In [6]:
# BASH COMMAND TO USE FOR dates.txt CREATION
# printf '%s\n' * | cut -d . -f2 >> dates.txt

with open('../data/SCS/MODIS_DATA_DAILY/dates.txt', 'rb') as file:
     data = file.read()


dates_list = data.decode("utf-16").split("\r\n")

In [6]:
chl["time"] = [datetime.datetime.strptime(i, "%Y%m%d") for i in dates_list]
chl = chl.interp(lon = lons, lat = lats, method = "nearest")

In [7]:
chl.load().to_netcdf("../data/SCS/MODIS_chl_extended.nc")

In [53]:
chl = xr.open_dataset("../data/SCS/MODIS_chl_extended.nc").convert_calendar("noleap")

datetimeindex = chl.indexes['time'].to_datetimeindex()

chl["time"] = datetimeindex

c:\Users\feder\miniconda3\lib\site-packages\xarray\coding\times.py:351: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
C:\Users\feder\AppData\Local\Temp\ipykernel_11768\1850234950.py:3: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  datetimeindex = chl.indexes['time'].to_datetimeindex()


In [55]:
chl.load().to_netcdf("../data/SCS/MODIS_chl_extended_TRIAL.nc")

# Computation of chlorophyll and temperature climatologies

In [18]:
chl = xr.open_dataset("../data/SCS/MODIS_chl_extended.nc").chlor_a
temp = xr.open_dataset("../data/SCS/CMEMS_temp_extended.nc").analysed_sst

lons, lats = temp.lon, temp.lat

In [19]:
chl_clima = chl.groupby("time.dayofyear").mean()

temp_clima = temp.groupby("time.dayofyear").mean()

In [20]:
chl_std = chl.groupby("time.dayofyear").std(skipna = True)

temp_std = temp.groupby("time.dayofyear").std(skipna = True)

c:\Users\feder\miniconda3\lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\feder\miniconda3\lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\feder\miniconda3\lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\feder\miniconda3\lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\feder\miniconda3\lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\feder\miniconda3\lib\site-packages\numpy\lib\n

In [21]:
days_list = [datetime.datetime.strptime(str(2016) + "-" + str(chl_clima.dayofyear.to_numpy()[i]), "%Y-%j").strftime("%d-%m") for i in range(0, 366)]

chl_clima["dayofyear"] = days_list
temp_clima["dayofyear"] = days_list

chl_std["dayofyear"] = days_list
temp_std["dayofyear"] = days_list

In [30]:
chl_clima.to_netcdf("../data/SCS/chl_climatology.nc")
chl_std.to_netcdf("../data/SCS/chl_std.nc")

In [ ]:
temp_clima.to_netcdf("../data/SCS/temp_climatology.nc")
temp_std.to_netcdf("../data/SCS/temp_std.nc")

# Bathymetry interpolation

In [ ]:
bathy = xr.open_dataset("../data/SCS/bathymetry.nc")

bathy = bathy.interp(lon = lons, lat = lats, method = "nearest")

bathy.to_netcdf("../data/SCS/bathymetry_interpolated.nc")

# Time uniformation for temperature

In [3]:
temp = xr.open_dataset("../data/SCS/CMEMS_temp_extended_standardtime.nc")#.analysed_sst
temp["time"] = temp["time"].dt.strftime("%Y-%m-%d").astype(np.datetime64)

temp.analysed_sst.load().to_netcdf("../data/SCS/CMEMS_temp_extended.nc")

# 8-day averaged time series

In [12]:
chlorophyll = xr.open_dataset("../data/SCS/MODIS_chl_extended.nc").convert_calendar("noleap")

chlorophyll = chlorophyll.sel(time=~((chlorophyll.time.dt.month == 12) & (chlorophyll.time.dt.day >= 27))).resample(time = "1D").asfreq()

c = chlorophyll.where(chlorophyll["time.year"] == 2003, drop = True).resample(time = "8D").mean()
for y in range(2004,2022):
    c = xr.merge([c, chlorophyll.where(chlorophyll["time.year"] == y, drop = True).resample(time = "8D").mean()])

c = c.chlor_a

datetimeindex = c.indexes['time'].to_datetimeindex()

c["time"] = datetimeindex

c:\Users\feder\miniconda3\lib\site-packages\xarray\coding\times.py:351: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
C:\Users\feder\AppData\Local\Temp\ipykernel_38460\1833828430.py:17: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  datetimeindex = c.indexes['time'].to_datetimeindex()


In [ ]:
c.load().to_netcdf("../data/SCS/MODIS_chl_extended_8D.nc")

In [7]:
temperature = xr.open_dataset("../data/SCS/CMEMS_temp_extended.nc").convert_calendar("noleap")

temperature = temperature.sel(time=~((temperature.time.dt.month == 12) & (temperature.time.dt.day >= 27))).resample(time = "1D").asfreq()

t = temperature.where(temperature["time.year"] == 2003, drop = True).resample(time = "8D").mean()
for y in range(2004,2022):
    t = xr.merge([t, temperature.where(temperature["time.year"] == y, drop = True).resample(time = "8D").mean()])

t = t.analysed_sst

In [8]:
datetimeindex = t.indexes['time'].to_datetimeindex()

t["time"] = datetimeindex

c:\Users\feder\miniconda3\lib\site-packages\xarray\coding\times.py:351: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
C:\Users\feder\AppData\Local\Temp\ipykernel_38460\2651879082.py:1: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  datetimeindex = t.indexes['time'].to_datetimeindex()


In [10]:
t.load().to_netcdf("../data/SCS/CMEMS_temp_extended_8D.nc")

In [4]:
# chl = xr.open_mfdataset("../data/SCS/MODIS_DATA_8D/*.nc", combine = "nested", concat_dim="time")

In [13]:
# BASH COMMAND TO USE FOR dates.txt CREATION
# printf '%s\n' * | cut -d . -f2 >> dates.txt

# with open('../data/SCS/MODIS_DATA_8D/dates.txt', 'rb') as file:
#      data = file.read()


# dates_list = data.decode("utf-16").split("\r\n")

In [16]:
# chl["time"] = [datetime.datetime.strptime(i, "%Y%m%d") for i in dates_list]
# chl = chl.interp(lon = lons, lat = lats, method = "nearest")

# chl.chlor_a.load().to_netcdf("../data/SCS/MODIS_chl_extended_8D.nc")